In [3]:
import pandas as pd #lets get our importing out of the way
import numpy as np

school_data = pd.read_csv("Resources/schools_complete.csv") #importing in as a dataframe
student_data = pd.read_csv("Resources/students_complete.csv")

In [4]:
school_data.columns, student_data.columns #see what we're working with

(Index(['School ID', 'school_name', 'type', 'size', 'budget'], dtype='object'),
 Index(['Student ID', 'student_name', 'gender', 'grade', 'school_name',
        'reading_score', 'math_score'],
       dtype='object'))

In [5]:
school_data.head() #closer look

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [6]:
student_data.head() #closer look

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [7]:
print(school_data.shape) #very small
student_data.shape # very large!

(15, 5)


(39170, 7)

In [8]:
district_df_titles = ["# of Schools", "# of Students", "Budget","Avg Math","Avg Reading", "% Passing Math", "% Passing Reading", "% Passing Both"]
# made the titles a list so hopefully it is easier to make the later df titles
district_df_values = []

In [9]:
school_count = len(school_data["School ID"].unique())
district_df_values.append(school_count)

In [10]:
student_count = len(student_data["Student ID"].unique())
district_df_values.append(student_count)


In [11]:
total_budget = school_data["budget"].sum()
district_df_values.append(total_budget)

In [12]:
avg_math = student_data["math_score"].mean()
district_df_values.append(avg_math)

In [13]:
avg_read = student_data["reading_score"].mean()
district_df_values.append(avg_read)

In [14]:
passing_math_perc = len(student_data[student_data["math_score"]>=70])*100/student_count 
# asumming 70% is the lowest passing score
district_df_values.append(passing_math_perc)

In [15]:
passing_read_perc = len(student_data[student_data["reading_score"]>=70])*100/student_count 
# asumming 70% is the lowest passing score
district_df_values.append(passing_read_perc)

In [16]:
passing_both_perc = len(student_data[(student_data["reading_score"]>=70) & (student_data["math_score"]>=70)])*100/student_count
# student_data["reading_score"]>=70 makes a df of trues and falses, the student_data[] outside makes a df of all the trues
# this is a long function, but just looks for where reading and math scores are both > 70, makes a df and counts them
district_df_values.append(passing_both_perc)

In [17]:
district_summary = pd.DataFrame([district_df_values], columns = district_df_titles) # has to be a list of lists to seperate into  a row


In [18]:
district_summary["# of Students"] = district_summary["# of Students"].map("{:,}".format)
district_summary["Budget"] = district_summary["Budget"].map("${:,.2f}".format)
district_summary
#Don't forget to clear values list before running again

,# of Schools,# of Students,Budget,Avg Math,Avg Reading,% Passing Math,% Passing Reading,% Passing Both
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


In [19]:
merged_school = pd.merge(school_data, student_data, how = "inner", on = "school_name")

In [20]:
merged_school

,School ID,school_name,type,size,budget,Student ID,student_name,gender,grade,reading_score,math_score
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84
...,...,...,...,...,...,...,...,...,...,...,...
39165,14,Thomas High School,Charter,1635,1043130,39165,Donna Howard,F,12th,99,90
39166,14,Thomas High School,Charter,1635,1043130,39166,Dawn Bell,F,10th,95,70
39167,14,Thomas High School,Charter,1635,1043130,39167,Rebecca Tanner,F,9th,73,84
39168,14,Thomas High School,Charter,1635,1043130,39168,Desiree Kidd,F,10th,99,90


In [21]:
school_name = list(school_data["school_name"].unique()) # School names
school_type = [school_data["type"]] #everything should be in order unless the csv is reorganized
school_values = []


In [22]:
school_types = school_data.set_index(["school_name"])["type"]
school_values.append(school_types)

In [23]:
per_school_counts = school_data.set_index(["school_name"])["size"]
school_values.append(per_school_counts)

In [24]:
per_school_budget = school_data.set_index(["school_name"])["budget"]
school_values.append(per_school_budget)

In [25]:
per_student_budget = per_school_budget/per_school_counts
school_values.append(per_student_budget)

In [26]:
avg_scores_school = merged_school.set_index(["school_name"])[["math_score","reading_score"]].groupby(["school_name"]).mean() # takes average math and reading
#i need to remember to list within list, personal hueristic: "multiple columns require multiple bracket sets"
school_values.append(avg_scores_school)

In [27]:
passing_math_students = merged_school[
                                        (merged_school["math_score"]>=70)]
passing_math_students = passing_math_students.set_index(["school_name"])["math_score"].groupby(["school_name"]).count()
passing_math_perc_school = passing_math_students*100 /per_school_counts
school_values.append(passing_math_perc_school)

In [28]:
passing_read_students = merged_school[
                                        (merged_school["reading_score"]>=70)]
passing_read_students = passing_read_students.set_index(["school_name"])["reading_score"].groupby(["school_name"]).count()
passing_read_perc_school = passing_read_students*100 /per_school_counts
school_values.append(passing_read_perc_school)

In [29]:
passing_both_students = merged_school[
                                        (merged_school["reading_score"]>=70) & (merged_school["math_score"]>=70)]
passing_both_students = passing_both_students.set_index(["school_name"])["reading_score"].groupby(["school_name"]).count()# don't need both columns. They will be the same.

passing_both_perc_school = passing_both_students*100 /per_school_counts
school_values.append(passing_both_perc_school)

In [30]:
school_info_df = pd.concat(school_values, axis = 1) # combine our dataframes. Axis 0 will stack, axis 1 will make horizontal
school_info_df.columns = ["type","size","Total Budget","Budget per Student", "Average Math Score","Average Reading Score",
                         "Percent Passing Math","Percent Passing Reading", "Percent Passing Both"]

school_info_df["Total Budget"] = school_info_df["Total Budget"] = school_info_df["Total Budget"].map("${:,.2f}".format)
#school_info_df["size"] = school_info_df["size"].map("{:,}".format) 
#Removed formatting for size and budget per student. the formats confuse the code later down the line
school_info_df

,type,size,Total Budget,Budget per Student,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Percent Passing Both
school_name,,,,,,,,,
Huang High School,District,2917,"$1,910,635.00",655.0,76.629414,81.182722,65.683922,81.316421,53.513884
Figueroa High School,District,2949,"$1,884,411.00",639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Shelton High School,Charter,1761,"$1,056,600.00",600.0,83.359455,83.725724,93.867121,95.854628,89.892107
Hernandez High School,District,4635,"$3,022,020.00",652.0,77.289752,80.934412,66.752967,80.862999,53.527508
Griffin High School,Charter,1468,"$917,500.00",625.0,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",578.0,83.274201,83.989488,93.867718,96.539641,90.582567
Cabrera High School,Charter,1858,"$1,081,356.00",582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Bailey High School,District,4976,"$3,124,928.00",628.0,77.048432,81.033963,66.680064,81.933280,54.642283
Holden High School,Charter,427,"$248,087.00",581.0,83.803279,83.814988,92.505855,96.252927,89.227166


In [31]:
top_schools = school_info_df.sort_values(by = ["Percent Passing Both"], ascending= False)
top_schools.head()

,type,size,Total Budget,Budget per Student,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Percent Passing Both
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",638.0,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",625.0,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",578.0,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",609.0,83.839917,84.044699,94.594595,95.945946,90.540541


In [32]:
bottom_schools = school_info_df.sort_values(by = ["Percent Passing Both"], ascending= True) 
bottom_schools.head()

,type,size,Total Budget,Budget per Student,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Percent Passing Both
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",637.0,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",655.0,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",652.0,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",650.0,77.072464,80.966394,66.057551,81.222432,53.539172


In [33]:
#Seperating by grade
ninth_graders = merged_school[(merged_school["grade"] == "9th")] #pulls out a df of only 9th graders
tenth_graders = merged_school[(merged_school["grade"] == "10th")]
eleventh_graders = merged_school[(merged_school["grade"] == "11th")]
twelfth_graders = merged_school[(merged_school["grade"] == "12th")]

math_scores_by_grade_list = []
ninth_grader_math_scores = ninth_graders.set_index(["school_name"])[["math_score"]].groupby(["school_name"]).mean()
math_scores_by_grade_list.append(ninth_grader_math_scores)

tenth_grader_math_scores = tenth_graders.set_index(["school_name"])[["math_score"]].groupby(["school_name"]).mean()
math_scores_by_grade_list.append(tenth_grader_math_scores)

eleventh_grader_math_scores = eleventh_graders.set_index(["school_name"])[["math_score"]].groupby(["school_name"]).mean()
math_scores_by_grade_list.append(eleventh_grader_math_scores)

twelfth_grader_math_scores = twelfth_graders.set_index(["school_name"])[["math_score"]].groupby(["school_name"]).mean()
math_scores_by_grade_list.append(twelfth_grader_math_scores)

math_scores_by_grade = pd.concat(math_scores_by_grade_list, axis= 1)

math_scores_by_grade.columns = ['9th','10th', '11th', '12th']
math_scores_by_grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [39]:
#run previous block first
read_scores_by_grade_list = []

ninth_grader_reading_scores = ninth_graders.set_index(["school_name"])[["reading_score"]].groupby(["school_name"]).mean()
read_scores_by_grade_list.append(ninth_grader_reading_scores)

tenth_grader_reading_scores = tenth_graders.set_index(["school_name"])[["reading_score"]].groupby(["school_name"]).mean()
read_scores_by_grade_list.append(tenth_grader_reading_scores)

eleventh_grader_reading_scores = eleventh_graders.set_index(["school_name"])[["reading_score"]].groupby(["school_name"]).mean()
read_scores_by_grade_list.append(eleventh_grader_reading_scores)

twelfth_grader_reading_scores = twelfth_graders.set_index(["school_name"])[["reading_score"]].groupby(["school_name"]).mean()
read_scores_by_grade_list.append(twelfth_grader_reading_scores)

read_scores_by_grade = pd.concat(read_scores_by_grade_list, axis= 1)

read_scores_by_grade.columns = ['9th','10th', '11th', '12th']
read_scores_by_grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [35]:
school_spending_df = school_info_df.copy() #to preserve our school info dataframe

spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

school_spending_df["Budget per Student"] = (pd.cut(school_spending_df["Budget per Student"], bins = spending_bins, labels = labels))


In [36]:
school_spending_df = school_info_df.copy() #to preserve our school info dataframe

spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

school_spending_df["Budget per Student"] = (pd.cut(school_spending_df["Budget per Student"], bins = spending_bins, labels = labels))

spending_math_scores = school_spending_df.groupby(["Budget per Student"], observed = True)["Average Math Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Budget per Student"], observed = True)["Average Reading Score"].mean()
spending_passing_math = school_spending_df.groupby(["Budget per Student"], observed = True)["Percent Passing Math"].mean()
spending_passing_reading = school_spending_df.groupby(["Budget per Student"], observed = True)["Percent Passing Reading"].mean()
overall_passing_spending = school_spending_df.groupby(["Budget per Student"], observed = True)["Percent Passing Both"].mean()

school_spending_df = pd.concat([spending_math_scores,spending_reading_scores,spending_passing_math,spending_passing_reading,overall_passing_spending], axis = 1)

school_spending_df

,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Percent Passing Both
Budget per Student,,,,,
<$585,83.455399,83.933814,93.460096,96.610877,90.369459
$585-630,81.899826,83.155286,87.133538,92.718205,81.418596
$630-645,78.518855,81.624473,73.484209,84.391793,62.857656
$645-680,76.997210,81.027843,66.164813,81.133951,53.526855


In [37]:
school_size_df = school_info_df.copy()

size_bins = [0, 1000, 2000, 5000]
size_labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
school_size_df["size"] = (pd.cut(school_info_df["size"], bins = size_bins, labels = size_labels) )

size_math_scores = school_size_df.groupby(["size"], observed = True)["Average Math Score"].mean()
size_reading_scores = school_size_df.groupby(["size"], observed = True)["Average Reading Score"].mean()
size_passing_math = school_size_df.groupby(["size"], observed = True)["Percent Passing Math"].mean()
size_passing_reading = school_size_df.groupby(["size"], observed = True)["Percent Passing Reading"].mean()
size_overall_passing = school_size_df.groupby(["size"], observed = True)["Percent Passing Both"].mean()


school_size_df = pd.concat([size_math_scores,size_reading_scores,size_passing_math,size_passing_reading,size_overall_passing], axis = 1)

school_size_df

,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Percent Passing Both
size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


In [38]:
type_summary = school_info_df.set_index(["type"]).loc[:,"Average Math Score":].groupby(["type"]).mean()

type_summary

,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Percent Passing Both
type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
